In [1]:
import numpy as np
import cvxpy as cp
import heapdict
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
import time

from pytransform3d.rotations import *

import open3d as o3d
import copy

In [2]:
def pc_to_np(open3d_data):
    return np.asarray(open3d_data.points)

def np_to_pc(np_data):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np_data)
    return pcd

def rotation_matrix(euler):
    R = np.identity(4)
    R[0:3,0:3] = matrix_from_euler_xyz(euler)
    return R

def scale_matrix(euler):
    S = np.identity(4)
    S[0,0] = euler[0]
    S[1,1] = euler[1]
    S[2,2] = euler[2]
    return S

def translation_matrix(euler):
    T = np.identity(4)
    T[0,3] = euler[0]
    T[1,3] = euler[1]
    T[2,3] = euler[2]
    return T

In [3]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

def icp(source, target, T_init, threshold=0.8):
    
    evaluation = o3d.registration.evaluate_registration(source, target, threshold, T_init)

    reg_p2p = o3d.registration.registration_icp(source, target, threshold, T_init, 
                                                o3d.registration.TransformationEstimationPointToPoint())
    
    return reg_p2p.transformation

In [4]:
def compute_dist(src,tgt,T):
    src_cpy = copy.deepcopy(src)
    src_t = src_cpy.transform(T)
    
    src_np = pc_to_np(src_t)
    tgt_np = pc_to_np(tgt)
    
    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(tgt_np)
    distances, indices = neigh.kneighbors(src_np, return_distance=True)
    return distances.ravel(), indices.ravel()

In [5]:
start = time.time()

src_ref = "data/model_bunny.txt"
tgt_ref = "data/data_bunny.txt"

T_iden = np.identity(4)

source = o3d.io.read_point_cloud(src_ref, format='xyz')
target = o3d.io.read_point_cloud(tgt_ref, format='xyz')

distances, indices = compute_dist(source,target,T_iden)
print('Initial: ' + str(np.sum(distances)))

source_vol = source.get_oriented_bounding_box().volume()
target_vol = target.get_oriented_bounding_box().volume()
source.scale(target_vol/source_vol)

distances, indices = compute_dist(source,target,T_iden)

s = source.compute_convex_hull()
t = target.compute_convex_hull()
source_hull = np_to_pc(np.asarray(s.vertices))
target_hull = np_to_pc(np.asarray(t.vertices))

src = source
tgt = target


p2p = np.identity(4)

ratio = 1.3
S = scale_matrix([ratio,ratio,ratio])
p2p = np.dot(S,p2p)




N = 5
for i in range(N):
    
    p2p = icp(src, tgt, p2p, threshold=0.8)
    distances, indices = compute_dist(src, tgt, p2p)
    print(np.sum(distances))
    
    #if(i != N-1):
        #R = rotation_matrix(np.random.rand(3))
        #p2p = np.dot(R, p2p)
        
        #src_centroid = src.get_center()
        #tgt_centroid = tgt.get_center()
        #T = translation_matrix(tgt_centroid - src_centroid)
        #p2p = np.dot(T, p2p)

print("Local registration took %.3f sec." % (time.time() - start))


distances, indices = compute_dist(source,target,p2p)
print(np.sum(distances))

print("Done.")

print(p2p)


Initial: 7524.494612795406
5972.435179087988
5202.737079078364
5165.013959576784
5165.035973264468
5165.019833079682
Local registration took 17.533 sec.
5165.019833079682
Done.
[[ 0.58505588  0.55155821  1.02151513  0.03037153]
 [-1.13843406  0.49658115  0.38389459 -0.07751173]
 [-0.22732688 -1.06732877  0.70649259 -0.34771775]
 [ 0.          0.          0.          1.        ]]


In [6]:
draw_registration_result(src,tgt,p2p)

In [9]:
src_vol = src.get_oriented_bounding_box().volume()
tgt_vol = tgt.get_oriented_bounding_box().volume()

p2p = np.identity(4)

R = rotation_matrix([0,1.57,0])
p2p = np.dot(R,p2p)

ratio = 1.3
S = scale_matrix([ratio,ratio,ratio])
p2p = np.dot(S,p2p)

T = translation_matrix([0.1,0.2,-0.1])
p2p = np.dot(T,p2p)

draw_registration_result(src,tgt,p2p)
distances, indices = compute_dist(source,target,p2p)
print(np.sum(distances))

4496.71913259495


In [74]:
src = o3d.io.read_point_cloud(src_ref, format='xyz')
tgt = o3d.io.read_point_cloud(tgt_ref, format='xyz')

R1 = np.identity(4)
p2p1 = icp(src, tgt, R1, threshold=0.8)
dist, _ = compute_dist(src,tgt,p2p1)
print(np.sum(dist))

R2 = np.identity(4)
p2p2 = icp(src, tgt, R2, threshold=0.8)
dist, _ = compute_dist(src,tgt,p2p2)
print(np.sum(dist))

5999.6275758902975
5999.6275758902975
